##  Clase 4: Proyecto Integrador – Gestión de Datos en Java y MySQL

**Duración total**: 4 horas (con pausas y práctica integrada)

---


###  **Objetivos de Aprendizaje**

* Aplicar colecciones y `Streams` en un sistema completo con base de datos MySQL.
* Diseñar e implementar una arquitectura por capas.
* Comprender ventajas y desventajas entre programación imperativa vs funcional (Streams).
* Ejecutar operaciones CRUD y consultas avanzadas.
* Analizar resultados y mejorar el diseño del código.

---


## Parte 1. Análisis de Requerimientos (30 min)

### Objetivo de esta sección:

---

###  1.1 Contexto del Proyecto

Sistema de gestión de productos y ventas para una tienda física o en línea.

**Funcionalidades básicas esperadas:**

* Registrar productos con información detallada.
* Gestionar usuarios (clientes y vendedores).
* Registrar ventas de productos.
* Realizar consultas y reportes de ventas.

---

### 1.2 Requerimientos funcionales

#### Producto

* Agregar nuevo producto.
* Listar productos.
* Buscar productos por nombre o categoría.
* Validar que el precio no sea negativo y que el nombre no esté vacío.
* Eliminar productos

#### Usuario

* Registrar usuario con nombre y tipo (cliente o vendedor).
* Actualizar usuario
* Consultar listado de usuarios.
* Eliminar usuario.


#### Venta

* Registrar una venta: se debe seleccionar producto, cliente, cantidad y vendedor.
* Calcular el total de ingresos de todas las ventas.
* Listar ventas por usuario o cliente.
* Obtener resumen de ventas por producto o por categoría.

---

### 1.3 Estructura de Datos (entidades y atributos)

Presenta esta estructura como una tabla o en forma de UML:

| Entidad  | Atributos                                                                                        |
| -------- | ------------------------------------------------------------------------------------------------ |
| Producto | id (int), nombre (String), categoría (String), precio (double)                                   |
| Usuario  | id (int), nombre (String), tipo (String: "cliente" o "vendedor")                                 |
| Venta    | id (int), idProducto (int), idCliente (int), idVendedor (int), cantidad (int), fecha (LocalDate) |

**Relaciones clave:**

* Un producto puede estar en muchas ventas.
* Un cliente puede tener muchas ventas.
* Un vendedor puede haber realizado muchas ventas.

---

### 1.4 Validaciones necesarias

* ¿Qué pasa si intento agregar un producto sin nombre?
* ¿Se debe permitir una venta sin producto o sin cliente?
* ¿Puede el precio de un producto ser negativo?
* ¿Cómo aseguramos que los IDs referencien entidades reales?

**Validaciones obligatorias sugeridas:**

| Entidad  | Campo                             | Validación             |
| -------- | --------------------------------- | ---------------------- |
| Producto | nombre                            | No vacío               |
| Producto | precio                            | > 0                    |
| Usuario  | tipo                              | "cliente" o "vendedor" |
| Venta    | cantidad                          | > 0                    |
| Venta    | idProducto, idCliente, idVendedor | Deben existir en la BD |

---

### 1.5 Operaciones del sistema (CRUD + Streams)

Haz una tabla o lista visual en el tablero con las operaciones principales:

| Entidad  | Operaciones CRUD                  | Consultas adicionales con Streams                           |
| -------- | --------------------------------- | ----------------------------------------------------------- |
| Producto | crear, leer, actualizar, eliminar | filtrar por categoría, ordenar por precio                   |
| Usuario  | crear, leer, actualizar, eliminar | agrupar por tipo                                            |
| Venta    | crear, leer, eliminar             | total ingresos, ventas por usuario, agrupación por producto |


---



### 1.6 Estructura de carpetas recomendada

```
proyecto/
├── config/
|   |__ VaadinConfig.java
├── models/
|   |__ Producto.java
│   ├── Usuario.java
│   └── Venta.java
├── repository/
│   ├── ProductoRepository.java
│   ├── UsuarioRepository.java
│   └── VentaRepository.java
├── service/
│   ├── ProductoService.java
│   ├── ReporteService.java
│   ├── UsuarioService.java
│   └── VentaService.java
├── view/
│   ├── HomeView.java
│   ├── MainLayoutView.java
│   ├── MainView.java
│   ├── ReporteView.java
│   ├── UsuarioView.java
│   └── VentaView.java
|
└── ProyectoApplication.java

main/
├── resources/
│   ├── application.properties
│   └── schema.sql


```

---


## Conexión, Diseño de la base de datos MySQL y pom.xml(10 min)


### 1. `application.properties` – conexión reutilizable

**Ubicación**: `resources/`
**Responsabilidad**: ofrecer una conexión lista para usar.


In [ ]:
# Database
spring.datasource.url=jdbc:mysql://localhost:3306/gestion_ventas?createDatabaseIfNotExist=true&useSSL=false&allowPublicKeyRetrieval=true
spring.datasource.username=root
spring.datasource.password=1126254560
spring.datasource.driver-class-name=com.mysql.cj.jdbc.Driver

# JPA
spring.jpa.show-sql=true
spring.jpa.hibernate.ddl-auto=update
spring.jpa.database-platform=org.hibernate.dialect.MySQL8Dialect
spring.jpa.properties.hibernate.format_sql=true

# Server
server.port=8081
server.error.whitelabel.enabled=false
server.servlet.context-path=/

# Vaadin
vaadin.whitelisted-packages=com.proyecto

---

### 2. `schema.sql` – creación de base y tablas

**Ubicación**: `resources/`
**Responsabilidad**: crear base de datos y tablas solo si no existen.



In [ ]:
CREATE DATABASE IF NOT EXISTS gestion_ventas;
USE gestion_ventas;

CREATE TABLE IF NOT EXISTS producto (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(100) NOT NULL,
    categoria VARCHAR(50),
    precio DOUBLE NOT NULL
);

CREATE TABLE IF NOT EXISTS usuario (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    nombre VARCHAR(100) NOT NULL,
    tipo VARCHAR(20) NOT NULL CHECK (tipo IN ('cliente', 'vendedor'))
);

CREATE TABLE IF NOT EXISTS venta (
    id BIGINT AUTO_INCREMENT PRIMARY KEY,
    producto_id BIGINT NOT NULL,
    cliente_id BIGINT NOT NULL,
    vendedor_id BIGINT NOT NULL,
    cantidad INT NOT NULL,
    fecha DATE NOT NULL,
    FOREIGN KEY (producto_id) REFERENCES producto(id),
    FOREIGN KEY (cliente_id) REFERENCES usuario(id),
    FOREIGN KEY (vendedor_id) REFERENCES usuario(id)
); 

### 3. `pom.xml` – Dependencias

**Ubicación**: `/`
**Responsabilidad**: instalación de dependencias del proyecto


In [ ]:
<?xml version="1.0" encoding="UTF-8"?>

<project xmlns="http://maven.apache.org/POM/4.0.0"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
  xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <parent>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-parent</artifactId>
    <version>3.1.9</version>
  </parent>

  <groupId>com.proyecto</groupId>
  <artifactId>proyecto-final</artifactId>
  <version>1.0-SNAPSHOT</version>
  <packaging>jar</packaging>

  <name>proyecto-final</name>
  <!-- FIXME change it to the project's website -->
  <url>http://www.example.com</url>

  <properties>
    <java.version>21</java.version>
    <vaadin.version>24.3.0</vaadin.version>
    <project.build.sourceEncoding>UTF-8</project.build.sourceEncoding>
  </properties>

  <repositories>
    <repository>
      <id>vaadin-addons</id>
      <url>https://maven.vaadin.com/vaadin-addons</url>
    </repository>
  </repositories>

  <dependencies>
    <!-- Spring Boot -->
    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-web</artifactId>
    </dependency>
    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-data-jpa</artifactId>
    </dependency>

    <!-- Vaadin -->
    <dependency>
      <groupId>com.vaadin</groupId>
      <artifactId>vaadin-spring-boot-starter</artifactId>
      <version>${vaadin.version}</version>
    </dependency>

    <!-- MySQL -->
    <dependency>
      <groupId>com.mysql</groupId>
      <artifactId>mysql-connector-j</artifactId>
      <version>8.0.33</version>
    </dependency>

    <!-- Lombok -->
    <dependency>
      <groupId>org.projectlombok</groupId>
      <artifactId>lombok</artifactId>
      <optional>true</optional>
    </dependency>

    <!-- Test -->
    <dependency>
      <groupId>org.springframework.boot</groupId>
      <artifactId>spring-boot-starter-test</artifactId>
      <scope>test</scope>
    </dependency>
  </dependencies>

  <build>
    <pluginManagement><!-- lock down plugins versions to avoid using Maven defaults (may be moved to
      parent pom) -->
      <plugins>
        <!-- clean lifecycle, see
        https://maven.apache.org/ref/current/maven-core/lifecycles.html#clean_Lifecycle -->
        <plugin>
          <artifactId>maven-clean-plugin</artifactId>
          <version>3.1.0</version>
        </plugin>
        <!-- default lifecycle, jar packaging: see
        https://maven.apache.org/ref/current/maven-core/default-bindings.html#Plugin_bindings_for_jar_packaging -->
        <plugin>
          <artifactId>maven-resources-plugin</artifactId>
          <version>3.0.2</version>
        </plugin>
        <plugin>
          <artifactId>maven-compiler-plugin</artifactId>
          <version>3.8.0</version>
        </plugin>
        <plugin>
          <artifactId>maven-surefire-plugin</artifactId>
          <version>2.22.1</version>
        </plugin>
        <plugin>
          <artifactId>maven-jar-plugin</artifactId>
          <version>3.0.2</version>
        </plugin>
        <plugin>
          <artifactId>maven-install-plugin</artifactId>
          <version>2.5.2</version>
        </plugin>
        <plugin>
          <artifactId>maven-deploy-plugin</artifactId>
          <version>2.8.2</version>
        </plugin>
        <!-- site lifecycle, see
        https://maven.apache.org/ref/current/maven-core/lifecycles.html#site_Lifecycle -->
        <plugin>
          <artifactId>maven-site-plugin</artifactId>
          <version>3.7.1</version>
        </plugin>
        <plugin>
          <artifactId>maven-project-info-reports-plugin</artifactId>
          <version>3.0.0</version>
        </plugin>
      </plugins>
    </pluginManagement>
    <plugins>
      <plugin>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-maven-plugin</artifactId>
        <version>3.2.3</version>
        <configuration>
          <excludes>
            <exclude>
              <groupId>org.projectlombok</groupId>
              <artifactId>lombok</artifactId>
            </exclude>
          </excludes>
        </configuration>
      </plugin>
    </plugins>
  </build>
</project>

## Configuración de `Vaadin` - Front

Vaadin es una plataforma de desarrollo de aplicaciones web, principalmente basada en Java, que facilita la creación de interfaces de usuario (UI) web sin necesidad de escribir HTML, CSS o JavaScript

**Ubicación**: `config/VaadinConfig.java`
**Responsabilidad**: Front el proyecto


In [ ]:
package com.proyecto.config;

import com.vaadin.flow.spring.annotation.EnableVaadin;
import org.springframework.context.annotation.Configuration;

@Configuration
@EnableVaadin
public class VaadinConfig {
}

## Capa `models`(45 min)


### Clase `Producto` (Modelo)

`models/Producto.java`



In [ ]:
// Define el paquete al que pertenece la clase Producto
package com.proyecto.models;

// Importa las anotaciones de JPA necesarias para mapear la clase a la base de datos
import jakarta.persistence.*;

// Anotaciones de Lombok para evitar escribir código repetitivo como getters, setters, constructores, etc.
import lombok.Data;
import lombok.NoArgsConstructor;
import lombok.AllArgsConstructor;

// Importa anotaciones específicas de Hibernate para configurar acciones de eliminación en cascada
import org.hibernate.annotations.OnDelete;
import org.hibernate.annotations.OnDeleteAction;

// Importa la clase List para manejar listas de objetos (ventas en este caso)
import java.util.List;

// Lombok: genera automáticamente getters(), setters(), toString(), equals(), hashCode()
@Data
// Lombok: genera un constructor sin argumentos
@NoArgsConstructor
// Lombok: genera un constructor con todos los atributos como argumentos
@AllArgsConstructor
// JPA: indica que esta clase es una entidad (se mapeará a una tabla en la base de datos)
@Entity
// JPA: especifica el nombre de la tabla que se usará en la base de datos
@Table(name = "producto")
public class Producto {

    // JPA: indica que este campo es la clave primaria
    @Id
    // JPA: indica que el valor del ID se genera automáticamente (auto-incremento)
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    // JPA: mapea la columna "nombre" y la marca como no nula (obligatoria)
    @Column(nullable = false)
    private String nombre;

    // JPA: mapea la columna "categoria" (puede ser nula)
    private String categoria;

    // JPA: mapea la columna "precio" y la marca como no nula
    @Column(nullable = false)
    private Double precio;

    // JPA: relación uno a muchos con la entidad Venta, la relación está mapeada desde el atributo "producto" en Venta
    // cascade = CascadeType.ALL: cualquier operación (persist, merge, remove...) se propaga a las ventas asociadas
    // orphanRemoval = true: si una venta se elimina de la lista, también se elimina de la base de datos
    @OneToMany(mappedBy = "producto", cascade = CascadeType.ALL, orphanRemoval = true)
    // Hibernate: asegura que al eliminar un producto también se eliminen sus ventas asociadas en la base de datos
    @OnDelete(action = OnDeleteAction.CASCADE)
    private List<Venta> ventas;

    // Constructor adicional (sin Lombok) que permite crear un producto sin asignar un ID manualmente
    public Producto(String nombre, String categoria, Double precio) {
        this.nombre = nombre;
        this.categoria = categoria;
        this.precio = precio;
    }

    // Sobrescribe el método toString para mostrar información legible del objeto Producto
    @Override
    public String toString() {
        return "Producto [id=" + id + ", nombre=" + nombre + ", categoria=" + categoria + ", precio=" + precio + "]";
    }
}


### Clase `Usuario` (Modelo)

`models/Usuario.java`



In [ ]:
// Define el paquete donde se encuentra esta clase
package com.proyecto.models;

// Importa anotaciones de JPA para el mapeo a la base de datos
import jakarta.persistence.*;

// Importa anotaciones de Lombok para generación automática de código (getters, setters, etc.)
import lombok.Data;
import lombok.NoArgsConstructor;
import lombok.AllArgsConstructor;

// Importa anotaciones de Hibernate para definir comportamiento al eliminar entidades relacionadas
import org.hibernate.annotations.OnDelete;
import org.hibernate.annotations.OnDeleteAction;

// Importa la clase List de Java para manejar colecciones
import java.util.List;

// Lombok: genera automáticamente getters, setters, toString(), equals() y hashCode()
@Data
// Lombok: genera un constructor sin argumentos
@NoArgsConstructor
// Lombok: genera un constructor con todos los campos como argumentos
@AllArgsConstructor
// JPA: indica que esta clase es una entidad JPA (tabla en base de datos)
@Entity
// JPA: especifica que esta entidad se mapeará a la tabla "usuario"
@Table(name = "usuario")
public class Usuario {

    // JPA: define el campo 'id' como la clave primaria de la tabla
    @Id
    // JPA: el valor del ID se generará automáticamente (autoincremental)
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    // JPA: define que la columna 'nombre' no puede ser nula
    @Column(nullable = false)
    private String nombre;

    // JPA: define que la columna 'tipo' no puede ser nula
    // Se usa para identificar si el usuario es un "cliente" o un "vendedor"
    @Column(nullable = false)
    private String tipo; // puede ser "cliente" o "vendedor"

    // JPA: relación uno a muchos entre Usuario y Venta cuando actúa como cliente
    // mappedBy: hace referencia al atributo 'cliente' en la clase Venta
    // cascade: cualquier operación se propaga a las ventas asociadas
    // orphanRemoval: si se elimina una venta de la lista, también se elimina de la base de datos
    @OneToMany(mappedBy = "cliente", cascade = CascadeType.ALL, orphanRemoval = true)
    // Hibernate: elimina las ventas asociadas en cascada cuando se borra el usuario
    @OnDelete(action = OnDeleteAction.CASCADE)
    private List<Venta> ventasComoCliente;

    // JPA: relación uno a muchos entre Usuario y Venta cuando actúa como vendedor
    @OneToMany(mappedBy = "vendedor", cascade = CascadeType.ALL, orphanRemoval = true)
    @OnDelete(action = OnDeleteAction.CASCADE)
    private List<Venta> ventasComoVendedor;

    // Constructor adicional sin usar Lombok, para inicializar un usuario con nombre y tipo
    public Usuario(String nombre, String tipo) {
        this.nombre = nombre;
        this.tipo = tipo;
    }

    // Método que retorna las ventas según el tipo de usuario
    public List<Venta> getVentas() {
        if (tipo.equals("cliente")) {
            return ventasComoCliente;
        } else {
            return ventasComoVendedor;
        }
    }

    // Getter manual para el ID
    public Long getId() {
        return id;
    }

    // Setter manual para el ID
    public void setId(Long id) {
        this.id = id;
    }

    // Getter manual para el nombre
    public String getNombre() {
        return nombre;
    }

    // Setter manual para el nombre
    public void setNombre(String nombre) {
        this.nombre = nombre;
    }

    // Getter manual para el tipo
    public String getTipo() {
        return tipo;
    }

    // Setter manual para el tipo, incluye validación para aceptar solo "cliente" o "vendedor"
    public void setTipo(String tipo) {
        if (!tipo.equals("cliente") && !tipo.equals("vendedor")) {
            throw new IllegalArgumentException("El tipo debe ser 'cliente' o 'vendedor'");
        }
        this.tipo = tipo;
    }

    // Sobrescribe el método toString para mostrar el usuario de forma legible
    @Override
    public String toString() {
        return tipo.toUpperCase() + ": " + nombre + " (ID: " + id + ")";
    }
}


### Clase `Venta` (Modelo)

`models/Venta.java`



In [ ]:
// Define el paquete en el que se encuentra esta clase
package com.proyecto.models;

// Importa las anotaciones necesarias de JPA para persistencia
import jakarta.persistence.*;

// Importa las anotaciones de Lombok para generar automáticamente métodos comunes
import lombok.Data;
import lombok.NoArgsConstructor;
import lombok.AllArgsConstructor;

// Importa la clase LocalDate para manejar fechas sin hora
import java.time.LocalDate;

// Lombok: genera automáticamente getters, setters, toString(), equals(), y hashCode()
@Data
// Lombok: genera un constructor sin argumentos
@NoArgsConstructor
// Lombok: genera un constructor con todos los campos como argumentos
@AllArgsConstructor
// JPA: indica que esta clase es una entidad que se mapeará a una tabla
@Entity
// JPA: especifica que esta entidad corresponde a la tabla "venta"
@Table(name = "venta")
public class Venta {

    // JPA: define el campo 'id' como la clave primaria de la tabla
    @Id
    // JPA: el valor del ID se generará automáticamente (estrategia autoincremental)
    @GeneratedValue(strategy = GenerationType.IDENTITY)
    private Long id;

    // JPA: relación muchos-a-uno con la entidad Producto
    // Cada venta está asociada a un solo producto
    @ManyToOne
    // JPA: define la columna "producto_id" como clave foránea no nula
    @JoinColumn(name = "producto_id", nullable = false)
    private Producto producto;

    // JPA: relación muchos-a-uno con la entidad Usuario actuando como cliente
    @ManyToOne
    @JoinColumn(name = "cliente_id", nullable = false)
    private Usuario cliente;

    // JPA: relación muchos-a-uno con la entidad Usuario actuando como vendedor
    @ManyToOne
    @JoinColumn(name = "vendedor_id", nullable = false)
    private Usuario vendedor;

    // JPA: columna que indica cuántos productos se vendieron, no puede ser nula
    @Column(nullable = false)
    private Integer cantidad;

    // JPA: columna que almacena la fecha de la venta, no puede ser nula
    @Column(nullable = false)
    private LocalDate fecha;

    // Constructor personalizado (sin usar Lombok) para inicializar campos clave de una venta
    public Venta(Producto producto, Usuario cliente, Usuario vendedor, Integer cantidad, LocalDate fecha) {
        this.producto = producto;
        this.cliente = cliente;
        this.vendedor = vendedor;
        this.cantidad = cantidad;
        this.fecha = fecha;
    }

    // Sobrescribe el método toString para mostrar un resumen legible de la venta
    @Override
    public String toString() {
        return "Venta #" + id + " | " + cantidad + " x " + producto.getNombre() + " a " +
                cliente.getNombre() + " por " + vendedor.getNombre() + " el " + fecha;
    }
}


## Capa `repository`(45 min)

**Spring Data JPA** 
Es un proyecto de Spring Framework que simplifica drásticamente el acceso a bases de datos relacionales mediante Java Persistence API (JPA). Automatiza gran parte del trabajo repetitivo necesario para interactuar con bases de datos y permite escribir consultas declarativas y limpias sin necesidad de mucho código.

¿Qué es JPA?
JPA (Java Persistence API) es una especificación estándar de Java para la persistencia de datos en bases de datos relacionales. No es una implementación en sí, sino una interfaz que implementan herramientas como:

* Hibernate (la más usada)
* EclipseLink
* OpenJPA

¿Qué hace Spring Data JPA?
Spring Data JPA:

* Elimina el código repetitivo del DAO o Repository tradicional.
* Proporciona consultas automáticas basadas en el nombre del método.
* Facilita el uso de JPQL, SQL nativo y consultas con criterios.
* Proporciona una implementación automática del repositorio.

`repository/ProductoRepository.java`



In [ ]:
// Define el paquete donde se encuentra esta interfaz
package com.proyecto.repository;

// Importa la clase Producto que representa la entidad que se va a gestionar
import com.proyecto.models.Producto;

// Importa la interfaz JpaRepository, que proporciona métodos CRUD y más funcionalidades JPA
import org.springframework.data.jpa.repository.JpaRepository;

// Importa la anotación @Repository para indicar que esta interfaz es un componente de repositorio
import org.springframework.stereotype.Repository;

// Anotación que marca esta interfaz como un repositorio de Spring (permite inyección de dependencias)
@Repository
// Define la interfaz ProductoRepository que extiende JpaRepository
// Esto permite realizar operaciones CRUD sobre la entidad Producto sin necesidad de implementar métodos
public interface ProductoRepository extends JpaRepository<Producto, Long> {
    // No es necesario escribir código adicional aquí a menos que se requieran consultas personalizadas
    // JpaRepository<Producto, Long> indica que se trabaja con la entidad Producto y su ID es de tipo Long
}


`repository/UsuarioRepository.java`

In [ ]:
// Define el paquete donde se encuentra esta interfaz
package com.proyecto.repository;

// Importa la entidad Usuario que será gestionada por este repositorio
import com.proyecto.models.Usuario;

// Importa JpaRepository que proporciona métodos CRUD y funcionalidades adicionales de JPA
import org.springframework.data.jpa.repository.JpaRepository;

// Importa la anotación @Repository para marcar la clase como un componente de repositorio de Spring
import org.springframework.stereotype.Repository;

// Anotación que indica que esta interfaz es un componente de acceso a datos de Spring (repositorio)
@Repository
// Se declara la interfaz UsuarioRepository que extiende JpaRepository
// Esto permite realizar operaciones CRUD y otras sobre la entidad Usuario sin escribir su implementación
public interface UsuarioRepository extends JpaRepository<Usuario, Long> {
    // JpaRepository<Usuario, Long> indica:
    // - Usuario: la entidad que manejará
    // - Long: el tipo de dato del campo ID de la entidad Usuario
    // No es necesario implementar métodos básicos; JpaRepository ya los proporciona
}


`repository/VentaRepository.java`

In [ ]:
// Define el paquete donde se encuentra esta interfaz
package com.proyecto.repository;

// Importa la entidad Venta, que representa las ventas en la base de datos
import com.proyecto.models.Venta;

// Importa JpaRepository, que proporciona los métodos CRUD y más funcionalidades de persistencia
import org.springframework.data.jpa.repository.JpaRepository;

// Importa la anotación @Repository, que indica que esta interfaz es un componente de repositorio en Spring
import org.springframework.stereotype.Repository;

// Marca esta interfaz como un repositorio, para que Spring la detecte y gestione como un componente
@Repository
// Define la interfaz VentaRepository que extiende JpaRepository
// JpaRepository<Venta, Long> especifica que:
// - Trabaja con la entidad Venta
// - El tipo del identificador (ID) de Venta es Long
public interface VentaRepository extends JpaRepository<Venta, Long> {
    // Al extender JpaRepository, esta interfaz hereda métodos como:
    // - findAll()
    // - findById(Long id)
    // - save(Venta venta)
    // - deleteById(Long id)
    // Puedes agregar aquí métodos personalizados si necesitas consultas específicas
}


## Capa `service`(45 min)

La **capa de servicio** (o **Service Layer**) en una arquitectura en capas **representa la lógica de negocio** de la aplicación. Actúa como intermediaria entre la **capa de controladores (controllers)** —que maneja las peticiones del usuario— y la **capa de acceso a datos (repositories o DAOs)** —que interactúa con la base de datos.
###  ¿Qué hace la capa de servicio?
#### 1. **Encapsula la lógica de negocio**: Evita que la lógica compleja esté en el controlador o directamente en los repositorios.
#### 2. **Orquesta múltiples operaciones**: Puede llamar varios métodos del repositorio, hacer validaciones, cálculos, conversiones, y luego retornar un resultado.
#### 3. **Promueve la reutilización de lógica**: Si varios controladores necesitan la misma operación, se centraliza aquí.
#### 4. **Facilita el mantenimiento y las pruebas**: La lógica de negocio separada permite pruebas unitarias más claras y código más limpio.


`service/ProductoService.java`



In [ ]:
// Define el paquete en el que se encuentra esta clase
package com.proyecto.service;

// Importa la entidad Producto
import com.proyecto.models.Producto;

// Importa el repositorio de Producto para acceder a la base de datos
import com.proyecto.repository.ProductoRepository;

// Marca esta clase como un componente de servicio de Spring
import org.springframework.stereotype.Service;

// Habilita la gestión de transacciones a nivel de clase
import org.springframework.transaction.annotation.Transactional;

// Importa la clase List para manejar listas de productos
import java.util.List;

// Marca esta clase como un servicio de Spring
@Service
// Anotación que indica que todos los métodos públicos están envueltos en una transacción
@Transactional
public class ProductoService {

    // Inyección del repositorio de productos como dependencia
    private final ProductoRepository repository;

    // Constructor que recibe el repositorio por inyección de dependencias
    public ProductoService(ProductoRepository repository) {
        this.repository = repository;
    }

    // Método para agregar un nuevo producto
    public void agregarProducto(String nombre, String categoria, double precio) {
        // Verifica que el nombre no sea nulo ni vacío
        if (nombre == null || nombre.trim().isEmpty()) {
            throw new IllegalArgumentException("El nombre del producto no puede estar vacío");
        }
        // Verifica que el precio no sea negativo
        if (precio < 0) {
            throw new IllegalArgumentException("El precio debe ser positivo");
        }

        // Crea una nueva instancia de Producto y la guarda en la base de datos
        Producto producto = new Producto(nombre.trim(), categoria, precio);
        repository.save(producto);
    }

    // Método que retorna la lista completa de productos desde la base de datos
    public List<Producto> listarProductos() {
        return repository.findAll();
    }

    // Método para obtener un producto por su ID
    public Producto obtenerProducto(Long id) {
        // Si el producto no se encuentra, lanza una excepción
        return repository.findById(id)
                .orElseThrow(() -> new IllegalArgumentException("Producto no encontrado"));
    }

    // Método para actualizar un producto existente
    public void actualizarProducto(Long id, String nombre, String categoria, double precio) {
        // Validación del nombre
        if (nombre == null || nombre.trim().isEmpty()) {
            throw new IllegalArgumentException("El nombre del producto no puede estar vacío");
        }
        // Validación del precio
        if (precio < 0) {
            throw new IllegalArgumentException("El precio debe ser positivo");
        }

        // Busca el producto por ID o lanza excepción si no existe
        Producto producto = repository.findById(id)
                .orElseThrow(() -> new IllegalArgumentException("Producto no encontrado"));

        // Actualiza los campos del producto
        producto.setNombre(nombre.trim());
        producto.setCategoria(categoria);
        producto.setPrecio(precio);

        // Guarda los cambios
        repository.save(producto);
    }

    // Método para eliminar un producto
    public void eliminarProducto(Long id) {
        // Busca el producto por ID o lanza excepción si no existe
        Producto producto = repository.findById(id)
                .orElseThrow(() -> new IllegalArgumentException("Producto no encontrado"));

        // Verifica si el producto tiene ventas asociadas
        if (producto.getVentas() != null && !producto.getVentas().isEmpty()) {
            throw new IllegalArgumentException("No se puede eliminar el producto porque tiene ventas asociadas");
        }

        // Elimina el producto de la base de datos
        repository.deleteById(id);
    }
}


`service/ReporteService.java`



In [ ]:
// Define el paquete en el que se encuentra esta clase
package com.proyecto.service;

// Importa la clase Venta que representa las ventas en el sistema
import com.proyecto.models.Venta;

// Marca esta clase como un servicio de Spring
import org.springframework.stereotype.Service;

// Importa las clases necesarias para operaciones estadísticas y de colecciones
import java.util.DoubleSummaryStatistics;
import java.util.List;
import java.util.Map;
import java.util.stream.Collectors;

// Marca esta clase como un servicio de Spring
@Service
public class ReporteService {

    // Método para generar un resumen de las ventas, incluyendo total, promedio y máximo
    public void resumenVentas(List<Venta> ventas) {
        // Usa el stream de Java para procesar la lista de ventas y calcular estadísticas
        DoubleSummaryStatistics stats = ventas.stream()
                .collect(Collectors.summarizingDouble(v -> v.getProducto().getPrecio() * v.getCantidad()));

        // Imprime el total de las ventas
        System.out.println("Total ventas: " + stats.getSum());
        // Imprime el promedio de las ventas
        System.out.println("Promedio: " + stats.getAverage());
        // Imprime el máximo valor de las ventas
        System.out.println("Máximo: " + stats.getMax());
    }

    // Método para generar un reporte de ventas por cada usuario (vendedor)
    public void ventasPorUsuario(List<Venta> ventas) {
        // Agrupa las ventas por el ID del vendedor y calcula el total por vendedor
        Map<Long, Double> porUsuario = ventas.stream()
                .collect(Collectors.groupingBy(
                        v -> v.getVendedor().getId(),  // Agrupa por el ID del vendedor
                        Collectors.summingDouble(v -> v.getProducto().getPrecio() * v.getCantidad())));  // Suma el valor total de las ventas

        // Imprime el total de ventas por cada vendedor
        porUsuario.forEach((id, total) -> System.out.println("Usuario ID: " + id + " - Total: $" + total));
    }

    // Método para filtrar y mostrar las ventas mayores a un valor umbral
    public void ventasMayoresA(double umbral, List<Venta> ventas) {
        // Filtra las ventas que superan el umbral y las imprime
        ventas.stream()
                .filter(v -> v.getProducto().getPrecio() * v.getCantidad() > umbral)  // Filtra por ventas mayores al umbral
                .forEach(System.out::println);  // Imprime cada venta que cumple la condición
    }
}


`service/UsuarioService.java`

In [ ]:
// Define el paquete en el que se encuentra esta clase
package com.proyecto.service;

// Importa las clases necesarias
import com.proyecto.models.Usuario;
import com.proyecto.repository.UsuarioRepository;
import org.springframework.stereotype.Service;
import org.springframework.transaction.annotation.Transactional;

// Importa las clases necesarias para manejar listas
import java.util.List;

// Marca esta clase como un servicio de Spring
@Service
// Marca esta clase como un servicio con soporte para transacciones
@Transactional
public class UsuarioService {
    // Inyecta el repositorio de usuarios para realizar operaciones de base de datos
    private final UsuarioRepository repository;

    // Constructor que inicializa el repositorio
    public UsuarioService(UsuarioRepository repository) {
        this.repository = repository;
    }

    // Método para registrar un nuevo usuario
    public void registrarUsuario(String nombre, String tipo) {
        // Verifica que el nombre no sea nulo o vacío
        if (nombre == null || nombre.isBlank()) {
            throw new IllegalArgumentException("El nombre no puede estar vacío");
        }
        // Verifica que el tipo de usuario sea 'cliente' o 'vendedor'
        if (!tipo.equals("cliente") && !tipo.equals("vendedor")) {
            throw new IllegalArgumentException("El tipo debe ser 'cliente' o 'vendedor'");
        }
        // Crea un nuevo objeto Usuario
        Usuario usuario = new Usuario(nombre, tipo);
        // Guarda el usuario en la base de datos
        repository.save(usuario);
    }

    // Método para actualizar un usuario existente
    public void actualizarUsuario(Usuario usuario) {
        // Verifica que el nombre del usuario no sea nulo o vacío
        if (usuario.getNombre() == null || usuario.getNombre().isBlank()) {
            throw new IllegalArgumentException("El nombre no puede estar vacío");
        }
        // Verifica que el tipo de usuario sea 'cliente' o 'vendedor'
        if (!usuario.getTipo().equals("cliente") && !usuario.getTipo().equals("vendedor")) {
            throw new IllegalArgumentException("El tipo debe ser 'cliente' o 'vendedor'");
        }
        // Guarda el usuario actualizado en la base de datos
        repository.save(usuario);
    }

    // Método para listar todos los usuarios
    public List<Usuario> listarUsuarios() {
        // Devuelve la lista de todos los usuarios desde el repositorio
        return repository.findAll();
    }

    // Método para eliminar un usuario
    public void eliminarUsuario(Long id) {
        // Busca el usuario por ID, lanza una excepción si no lo encuentra
        Usuario usuario = repository.findById(id)
                .orElseThrow(() -> new IllegalArgumentException("Usuario no encontrado"));

        // Verifica si el usuario tiene ventas asociadas
        if (usuario.getVentas() != null && !usuario.getVentas().isEmpty()) {
            throw new IllegalArgumentException("No se puede eliminar el usuario porque tiene ventas asociadas");
        }

        // Elimina el usuario de la base de datos
        repository.deleteById(id);
    }
}


`service/VentaService.java`

In [ ]:
// Define el paquete en el que se encuentra esta clase
package com.proyecto.service;

// Importa las clases necesarias
import com.proyecto.models.Venta;
import com.proyecto.models.Producto;
import com.proyecto.models.Usuario;
import com.proyecto.repository.VentaRepository;
import com.proyecto.repository.UsuarioRepository;
import org.springframework.stereotype.Service;
import org.springframework.transaction.annotation.Transactional;

import java.time.LocalDate;
import java.util.List;

// Marca esta clase como un servicio de Spring
@Service
// Marca esta clase como un servicio con soporte para transacciones
@Transactional
public class VentaService {
    // Declara las dependencias del repositorio de ventas y usuarios
    private final VentaRepository repository;
    private final UsuarioRepository usuarioRepository;

    // Constructor que inicializa los repositorios
    public VentaService(VentaRepository repository, UsuarioRepository usuarioRepository) {
        this.repository = repository;
        this.usuarioRepository = usuarioRepository;
    }

    // Método para registrar una venta
    public void registrarVenta(Producto producto, Usuario cliente, Usuario vendedor, int cantidad) {
        // Verifica que el producto no sea nulo
        if (producto == null) {
            throw new IllegalArgumentException("El producto no puede ser nulo");
        }
        // Verifica que el cliente no sea nulo
        if (cliente == null) {
            throw new IllegalArgumentException("El cliente no puede ser nulo");
        }
        // Verifica que el vendedor no sea nulo
        if (vendedor == null) {
            throw new IllegalArgumentException("El vendedor no puede ser nulo");
        }
        // Verifica que la cantidad sea mayor a 0
        if (cantidad <= 0) {
            throw new IllegalArgumentException("La cantidad debe ser mayor a 0");
        }

        // Verifica que el cliente exista en la base de datos
        Usuario clienteDB = usuarioRepository.findById(cliente.getId())
                .orElseThrow(() -> new IllegalArgumentException("El cliente no existe en la base de datos"));

        // Verifica que el vendedor exista en la base de datos
        Usuario vendedorDB = usuarioRepository.findById(vendedor.getId())
                .orElseThrow(() -> new IllegalArgumentException("El vendedor no existe en la base de datos"));

        // Verifica que el cliente tenga el tipo correcto (cliente)
        if (!clienteDB.getTipo().equals("cliente")) {
            throw new IllegalArgumentException("El usuario seleccionado no es un cliente");
        }
        // Verifica que el vendedor tenga el tipo correcto (vendedor)
        if (!vendedorDB.getTipo().equals("vendedor")) {
            throw new IllegalArgumentException("El usuario seleccionado no es un vendedor");
        }

        // Crea una nueva instancia de la venta con la fecha actual
        Venta venta = new Venta(producto, clienteDB, vendedorDB, cantidad, LocalDate.now());

        // Guarda la venta en la base de datos
        repository.save(venta);
    }

    // Método para listar todas las ventas
    public List<Venta> listarVentas() {
        // Devuelve la lista de todas las ventas desde el repositorio
        return repository.findAll();
    }

    // Método para eliminar una venta por su ID
    public void eliminarVenta(Long id) {
        // Elimina la venta de la base de datos por su ID
        repository.deleteById(id);
    }
}


## Capa `view`(45 min)

Proporciona la vista de cada parte de la aplicación

`view/HomeView.java`

In [ ]:
// Define el paquete donde se encuentra esta clase
package com.proyecto.view;

// Importa las clases necesarias de Vaadin para crear componentes de la interfaz de usuario
import com.vaadin.flow.component.html.H1;
import com.vaadin.flow.component.html.Paragraph;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.router.Route;
import org.springframework.stereotype.Component;

// Marca esta clase como un componente de Spring y como una vista de Vaadin
@Component
// Define la ruta para acceder a esta vista, y la asigna al layout principal
@Route(value = "", layout = MainLayout.class)
public class HomeView extends VerticalLayout {

    // Constructor de la clase, se ejecuta al crear la vista
    public HomeView() {
        // Ajusta el tamaño de la vista para que ocupe toda la pantalla
        setSizeFull();
        
        // Alinea los elementos al centro tanto vertical como horizontalmente
        setAlignItems(Alignment.CENTER);
        setJustifyContentMode(JustifyContentMode.CENTER);

        // Crea un título principal con un texto
        H1 title = new H1("Bienvenido a la aplicación de gestión de ventas");
        
        // Crea un párrafo con una breve descripción
        Paragraph description = new Paragraph(
                "Utilice el menú lateral para navegar entre las diferentes secciones del sistema.");

        // Añade el título y el párrafo a la vista
        add(title, description);
    }
}


`view/MainLayout.java.java`

In [ ]:
// Define el paquete donde se encuentra esta clase
package com.proyecto.view;

// Importa las clases necesarias de Vaadin para crear el layout de la aplicación
import com.vaadin.flow.component.applayout.AppLayout;
import com.vaadin.flow.component.applayout.DrawerToggle;
import com.vaadin.flow.component.html.H1;
import com.vaadin.flow.component.orderedlayout.FlexComponent;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.router.RouterLink;
import com.vaadin.flow.theme.lumo.LumoUtility;

// Clase principal que extiende de AppLayout, que es un layout comúnmente usado en aplicaciones con un menú lateral
public class MainLayout extends AppLayout {

    // Constructor de la clase, se ejecuta al crear la vista
    public MainLayout() {
        // Llama a los métodos para crear el encabezado y el menú lateral
        createHeader();
        createDrawer();
    }

    // Método que crea el encabezado de la aplicación (por ejemplo, el logo y el botón de menú)
    private void createHeader() {
        // Crea un título H1 con el texto "Sistema de Ventas"
        H1 logo = new H1("Sistema de Ventas");
        
        // Añade clases para modificar el estilo del logo (tamaño de fuente y márgenes)
        logo.addClassNames(
                LumoUtility.FontSize.LARGE,    // Aumenta el tamaño de la fuente
                LumoUtility.Margin.MEDIUM);    // Añade un margen medio

        // Crea un layout horizontal para el encabezado, que contiene el botón del menú (DrawerToggle) y el logo
        var header = new HorizontalLayout(new DrawerToggle(), logo);

        // Establece la alineación vertical de los componentes dentro del layout al centro
        header.setDefaultVerticalComponentAlignment(FlexComponent.Alignment.CENTER);
        // Expande el logo para que ocupe todo el espacio disponible en el layout horizontal
        header.expand(logo);
        // Establece el ancho total del encabezado
        header.setWidthFull();
        // Añade clases para modificar los márgenes y el padding
        header.addClassNames(
                LumoUtility.Padding.Vertical.NONE,   // Elimina el padding vertical
                LumoUtility.Padding.Horizontal.MEDIUM); // Añade padding horizontal medio

        // Añade el encabezado a la barra de navegación (navbar)
        addToNavbar(header);
    }

    // Método que crea el menú lateral (drawer) con los enlaces a diferentes vistas
    private void createDrawer() {
        // Crea enlaces (RouterLink) a las diferentes vistas de la aplicación
        RouterLink productosLink = new RouterLink("Productos", MainView.class);
        RouterLink ventasLink = new RouterLink("Ventas", VentaView.class);
        RouterLink usuariosLink = new RouterLink("Usuarios", UsuarioView.class);
        RouterLink reportesLink = new RouterLink("Reportes", ReporteView.class);

        // Añade los enlaces al menú lateral (drawer) usando un VerticalLayout
        addToDrawer(new VerticalLayout(
                productosLink,  // Enlace a la vista de productos
                ventasLink,     // Enlace a la vista de ventas
                usuariosLink,   // Enlace a la vista de usuarios
                reportesLink)); // Enlace a la vista de reportes
    }
}


`view/MainView.java`

In [ ]:
// Define el paquete donde se encuentra esta clase
package com.proyecto.view;

// Importa las clases necesarias de Vaadin para construir la vista
import com.proyecto.models.Producto;
import com.proyecto.service.ProductoService;
import com.vaadin.flow.component.button.Button;
import com.vaadin.flow.component.grid.Grid;
import com.vaadin.flow.component.notification.Notification;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.component.textfield.TextField;
import com.vaadin.flow.router.Route;
import org.springframework.stereotype.Component;

// Anota la clase como componente de Spring y define la ruta de navegación "productos"
@Component
@Route(value = "productos", layout = MainLayout.class)
public class MainView extends VerticalLayout {

    // Declara los componentes necesarios para la vista
    private final ProductoService productoService;
    private final Grid<Producto> grid = new Grid<>(Producto.class);
    private final TextField nombreField = new TextField("Nombre");
    private final TextField categoriaField = new TextField("Categoría");
    private final TextField precioField = new TextField("Precio");
    private Producto productoSeleccionado = null;

    // Constructor que recibe el servicio de productos y configura la vista
    public MainView(ProductoService productoService) {
        this.productoService = productoService;

        setSizeFull();  // Establece el tamaño completo para la vista
        configureGrid(); // Configura el grid de productos
        add(grid, createFormLayout()); // Agrega el grid y el formulario a la vista
        refreshGrid(); // Refresca el grid con los productos
    }

    // Método que configura el grid (tabla) de productos
    private void configureGrid() {
        // Define las columnas a mostrar en el grid
        grid.setColumns("id", "nombre", "categoria", "precio");
        // Ajusta automáticamente el tamaño de las columnas
        grid.getColumns().forEach(col -> col.setAutoWidth(true));
        // Agrega un listener para detectar cuando se selecciona un producto en el grid
        grid.asSingleSelect().addValueChangeListener(event -> {
            productoSeleccionado = event.getValue();
            // Si hay un producto seleccionado, llena los campos del formulario
            if (productoSeleccionado != null) {
                nombreField.setValue(productoSeleccionado.getNombre());
                categoriaField.setValue(productoSeleccionado.getCategoria());
                precioField.setValue(String.valueOf(productoSeleccionado.getPrecio()));
            } else {
                // Si no hay selección, limpia los campos del formulario
                clearFields();
            }
        });
    }

    // Método que crea el formulario con los campos y botones para agregar, modificar o eliminar productos
    private HorizontalLayout createFormLayout() {
        // Botón para agregar un nuevo producto
        Button addButton = new Button("Agregar Producto", e -> {
            try {
                // Validaciones para asegurarse de que los campos no estén vacíos
                if (nombreField.getValue() == null || nombreField.getValue().trim().isEmpty()) {
                    Notification.show("El nombre del producto no puede estar vacío", 3000,
                            Notification.Position.MIDDLE);
                    return;
                }
                if (categoriaField.getValue() == null || categoriaField.getValue().trim().isEmpty()) {
                    Notification.show("La categoría del producto no puede estar vacía", 3000,
                            Notification.Position.MIDDLE);
                    return;
                }
                if (precioField.getValue() == null || precioField.getValue().trim().isEmpty()) {
                    Notification.show("El precio del producto no puede estar vacío", 3000,
                            Notification.Position.MIDDLE);
                    return;
                }
                double precio = Double.parseDouble(precioField.getValue());
                // Validación para asegurarse de que el precio sea positivo
                if (precio < 0) {
                    Notification.show("El precio no puede ser negativo", 3000, Notification.Position.MIDDLE);
                    return;
                }
                // Crear un nuevo producto y agregarlo al servicio
                Producto producto = new Producto(
                        nombreField.getValue(),
                        categoriaField.getValue(),
                        precio);
                productoService.agregarProducto(
                        producto.getNombre(),
                        producto.getCategoria(),
                        producto.getPrecio());
                refreshGrid(); // Refresca la lista de productos en el grid
                clearFields(); // Limpia los campos del formulario
                Notification.show("Producto agregado exitosamente", 3000, Notification.Position.MIDDLE);
            } catch (NumberFormatException ex) {
                Notification.show("El precio debe ser un número válido", 3000, Notification.Position.MIDDLE);
            } catch (IllegalArgumentException ex) {
                Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);
            }
        });

        // Botón para modificar un producto seleccionado
        Button updateButton = new Button("Modificar Producto", e -> {
            if (productoSeleccionado != null) {
                try {
                    // Validaciones para asegurarse de que los campos no estén vacíos
                    if (nombreField.getValue() == null || nombreField.getValue().trim().isEmpty()) {
                        Notification.show("El nombre del producto no puede estar vacío", 3000,
                                Notification.Position.MIDDLE);
                        return;
                    }
                    if (categoriaField.getValue() == null || categoriaField.getValue().trim().isEmpty()) {
                        Notification.show("La categoría del producto no puede estar vacía", 3000,
                                Notification.Position.MIDDLE);
                        return;
                    }
                    if (precioField.getValue() == null || precioField.getValue().trim().isEmpty()) {
                        Notification.show("El precio del producto no puede estar vacío", 3000,
                                Notification.Position.MIDDLE);
                        return;
                    }
                    double precio = Double.parseDouble(precioField.getValue());
                    // Validación para asegurarse de que el precio sea positivo
                    if (precio < 0) {
                        Notification.show("El precio no puede ser negativo", 3000, Notification.Position.MIDDLE);
                        return;
                    }
                    // Actualiza el producto en el servicio
                    productoService.actualizarProducto(
                            productoSeleccionado.getId(),
                            nombreField.getValue(),
                            categoriaField.getValue(),
                            precio);
                    refreshGrid(); // Refresca la lista de productos en el grid
                    clearFields(); // Limpia los campos del formulario
                    Notification.show("Producto modificado exitosamente", 3000, Notification.Position.MIDDLE);
                } catch (NumberFormatException ex) {
                    Notification.show("El precio debe ser un número válido", 3000, Notification.Position.MIDDLE);
                } catch (IllegalArgumentException ex) {
                    Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);
                }
            } else {
                Notification.show("Por favor seleccione un producto para modificar", 3000,
                        Notification.Position.MIDDLE);
            }
        });

        // Botón para eliminar el producto seleccionado
        Button deleteButton = new Button("Eliminar Producto", e -> {
            if (productoSeleccionado != null) {
                try {
                    // Elimina el producto y refresca el grid
                    productoService.eliminarProducto(productoSeleccionado.getId());
                    refreshGrid();
                    clearFields(); // Limpia los campos del formulario
                    Notification.show("Producto eliminado exitosamente", 3000, Notification.Position.MIDDLE);
                } catch (IllegalArgumentException ex) {
                    Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);
                }
            } else {
                Notification.show("Por favor seleccione un producto para eliminar", 3000, Notification.Position.MIDDLE);
            }
        });

        // Devuelve el layout horizontal con los campos y botones
        return new HorizontalLayout(nombreField, categoriaField, precioField, addButton, updateButton, deleteButton);
    }

    // Método que refresca la lista de productos en el grid
    private void refreshGrid() {
        grid.setItems(productoService.listarProductos());
    }

    // Método que limpia los campos del formulario
    private void clearFields() {
        nombreField.clear();
        categoriaField.clear();
        precioField.clear();
        productoSeleccionado = null;
    }
}


`view/ReporteView.java`

In [ ]:
package com.proyecto.view;

// Importación de las clases necesarias para el proyecto
import com.proyecto.models.Venta;
import com.proyecto.models.Usuario;
import com.proyecto.service.VentaService;
import com.proyecto.service.UsuarioService;
import com.vaadin.flow.component.button.Button;
import com.vaadin.flow.component.grid.Grid;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.component.textfield.TextField;
import com.vaadin.flow.router.Route;
import org.springframework.stereotype.Component;

import java.util.DoubleSummaryStatistics;
import java.util.List;
import java.util.Map;
import java.util.stream.Collectors;

// Declaración de la clase como un componente de Vaadin y la ruta para acceder a esta vista
@Component
@Route(value = "reportes", layout = MainLayout.class)
public class ReporteView extends VerticalLayout {

    // Servicios necesarios para obtener los datos de ventas y usuarios
    private final VentaService ventaService;
    private final UsuarioService usuarioService;

    // Definición del grid para mostrar las ventas
    private final Grid<Venta> grid = new Grid<>(Venta.class);

    // Campos de texto para mostrar estadísticas
    private final TextField totalVentasField = new TextField("Total Ventas");
    private final TextField promedioVentasField = new TextField("Promedio por Venta");
    private final TextField ventaMaximaField = new TextField("Venta Máxima");
    private final TextField ventaMinimaField = new TextField("Venta Mínima");

    // Constructor de la clase, donde se inicializan los servicios y la vista
    public ReporteView(VentaService ventaService, UsuarioService usuarioService) {
        this.ventaService = ventaService;
        this.usuarioService = usuarioService;

        // Configuración del tamaño de la vista
        setSizeFull();

        // Configuración de los elementos de la vista
        setupView();

        // Actualizar las estadísticas al cargar la vista
        actualizarEstadisticas();
    }

    // Configuración de la vista, incluyendo la creación del grid y los botones
    private void setupView() {
        // Configuración de las columnas del grid
        grid.setColumns("id", "producto", "cliente", "vendedor", "cantidad", "fecha");

        // Asegurar que todas las columnas tengan un ancho automático
        grid.getColumns().forEach(col -> col.setAutoWidth(true));

        // Configuración de los campos de estadísticas, que serán de solo lectura
        totalVentasField.setReadOnly(true);
        promedioVentasField.setReadOnly(true);
        ventaMaximaField.setReadOnly(true);
        ventaMinimaField.setReadOnly(true);

        // Crear botones para navegar a otras vistas
        Button productosButton = new Button("Ir a Productos", e -> {
            getUI().ifPresent(ui -> ui.navigate("productos"));
        });

        Button ventasButton = new Button("Ir a Ventas", e -> {
            getUI().ifPresent(ui -> ui.navigate("ventas"));
        });

        Button usuariosButton = new Button("Ir a Usuarios", e -> {
            getUI().ifPresent(ui -> ui.navigate("usuarios"));
        });

        // Botones para mostrar las ventas por vendedor o cliente
        Button ventasPorVendedorButton = new Button("Ventas por Vendedor", e -> {
            mostrarVentasPorVendedor();
        });

        Button ventasPorClienteButton = new Button("Ventas por Cliente", e -> {
            mostrarVentasPorCliente();
        });

        // Botón para actualizar los reportes
        Button refreshButton = new Button("Actualizar Reportes", e -> {
            actualizarEstadisticas();  // Actualiza las estadísticas
            grid.setItems(ventaService.listarVentas());  // Actualiza los elementos del grid
        });

        // Crear una barra horizontal para los botones de navegación
        HorizontalLayout header = new HorizontalLayout(productosButton, ventasButton, usuariosButton);

        // Crear una barra horizontal para las estadísticas
        HorizontalLayout statsLayout = new HorizontalLayout(
                totalVentasField, promedioVentasField, ventaMaximaField, ventaMinimaField);

        // Crear una barra horizontal para los botones de acción
        HorizontalLayout buttonsLayout = new HorizontalLayout(
                ventasPorVendedorButton, ventasPorClienteButton, refreshButton);

        // Agregar las diferentes partes a la vista
        add(header, statsLayout, buttonsLayout, grid);

        // Establecer los elementos del grid con la lista de ventas
        grid.setItems(ventaService.listarVentas());
    }

    // Método para actualizar las estadísticas basadas en las ventas
    private void actualizarEstadisticas() {
        // Obtener la lista de ventas
        List<Venta> ventas = ventaService.listarVentas();

        // Calcular las estadísticas de las ventas (suma, promedio, máxima, mínima)
        DoubleSummaryStatistics stats = ventas.stream()
                .collect(Collectors.summarizingDouble(v -> v.getProducto().getPrecio() * v.getCantidad()));

        // Mostrar las estadísticas en los campos correspondientes
        totalVentasField.setValue(String.format("$%.2f", stats.getSum()));
        promedioVentasField.setValue(String.format("$%.2f", stats.getAverage()));
        ventaMaximaField.setValue(String.format("$%.2f", stats.getMax()));
        ventaMinimaField.setValue(String.format("$%.2f", stats.getMin()));
    }

    // Método para mostrar las ventas agrupadas por vendedor
    private void mostrarVentasPorVendedor() {
        // Obtener la lista de ventas
        List<Venta> ventas = ventaService.listarVentas();

        // Agrupar las ventas por vendedor y calcular el total de ventas por vendedor
        Map<Usuario, Double> ventasPorVendedor = ventas.stream()
                .collect(Collectors.groupingBy(
                        Venta::getVendedor,
                        Collectors.summingDouble(v -> v.getProducto().getPrecio() * v.getCantidad())));

        // Crear un reporte con las ventas por vendedor
        StringBuilder reporte = new StringBuilder("Ventas por Vendedor:\n");
        ventasPorVendedor.forEach(
                (vendedor, total) -> reporte.append(String.format("%s: $%.2f\n", vendedor.getNombre(), total)));

        // Ordenar las ventas por nombre del vendedor y actualizar el grid
        grid.setItems(ventas.stream()
                .sorted((v1, v2) -> v1.getVendedor().getNombre().compareTo(v2.getVendedor().getNombre()))
                .toList());
    }

    // Método para mostrar las ventas agrupadas por cliente
    private void mostrarVentasPorCliente() {
        // Obtener la lista de ventas
        List<Venta> ventas = ventaService.listarVentas();

        // Agrupar las ventas por cliente y calcular el total de ventas por cliente
        Map<Usuario, Double> ventasPorCliente = ventas.stream()
                .collect(Collectors.groupingBy(
                        Venta::getCliente,
                        Collectors.summingDouble(v -> v.getProducto().getPrecio() * v.getCantidad())));

        // Crear un reporte con las ventas por cliente
        StringBuilder reporte = new StringBuilder("Ventas por Cliente:\n");
        ventasPorCliente
                .forEach((cliente, total) -> reporte.append(String.format("%s: $%.2f\n", cliente.getNombre(), total)));

        // Ordenar las ventas por nombre del cliente y actualizar el grid
        grid.setItems(ventas.stream()
                .sorted((v1, v2) -> v1.getCliente().getNombre().compareTo(v2.getCliente().getNombre()))
                .toList());
    }
}


`view/UsuarioView.java`

In [ ]:
package com.proyecto.view;  // Define el paquete de la clase en el proyecto.

import com.proyecto.models.Usuario;  // Importa la clase Usuario desde el paquete de modelos.
import com.proyecto.service.UsuarioService;  // Importa el servicio para manejar la lógica de los usuarios.
import com.vaadin.flow.component.button.Button;  // Importa el componente Button de Vaadin.
import com.vaadin.flow.component.combobox.ComboBox;  // Importa el componente ComboBox de Vaadin.
import com.vaadin.flow.component.grid.Grid;  // Importa el componente Grid para mostrar datos tabulares.
import com.vaadin.flow.component.notification.Notification;  // Importa el componente de notificación de Vaadin.
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;  // Importa el componente HorizontalLayout para organizar los componentes en línea.
import com.vaadin.flow.component.orderedlayout.VerticalLayout;  // Importa el componente VerticalLayout para organizar los componentes en columna.
import com.vaadin.flow.component.textfield.TextField;  // Importa el componente TextField de Vaadin.
import com.vaadin.flow.router.Route;  // Importa la anotación Route para definir rutas en Vaadin.
import org.springframework.stereotype.Component;  // Importa la anotación Component de Spring para marcar la clase como un bean.

@Component  // Marca la clase como un componente gestionado por Spring.
@Route(value = "usuarios", layout = MainLayout.class)  // Define la ruta de la vista y la relación con el layout principal.
public class UsuarioView extends VerticalLayout {  // Define la clase UsuarioView que hereda de VerticalLayout para organizar los componentes en una columna.

    private final UsuarioService usuarioService;  // Declara el servicio de usuario, utilizado para realizar operaciones con los datos.
    private final Grid<Usuario> grid = new Grid<>(Usuario.class);  // Declara un Grid que mostrará los usuarios.
    private final TextField nombreField = new TextField("Nombre");  // Declara un campo de texto para el nombre del usuario.
    private final ComboBox<String> tipoField = new ComboBox<>("Tipo");  // Declara un ComboBox para seleccionar el tipo de usuario (cliente o vendedor).
    private Usuario usuarioSeleccionado = null;  // Declara una variable para almacenar el usuario seleccionado en el Grid.

    public UsuarioView(UsuarioService usuarioService) {  // Constructor de la clase, recibe un UsuarioService.
        this.usuarioService = usuarioService;  // Asigna el servicio de usuario.

        setSizeFull();  // Establece el tamaño completo de la vista.
        configureGrid();  // Configura el Grid de usuarios.
        configureForm();  // Configura el formulario de entrada de datos.
        add(createHeader(), grid, createFormLayout());  // Agrega el encabezado, el Grid y el formulario a la vista.
        refreshGrid();  // Refresca el Grid con los datos actuales de los usuarios.
    }

    private void configureGrid() {  // Método para configurar el Grid.
        grid.setColumns("id", "nombre", "tipo");  // Define las columnas que se mostrarán en el Grid.
        grid.getColumns().forEach(col -> col.setAutoWidth(true));  // Establece que las columnas se ajusten automáticamente al contenido.
        grid.asSingleSelect().addValueChangeListener(event -> {  // Añade un listener para detectar la selección de un usuario en el Grid.
            usuarioSeleccionado = event.getValue();  // Obtiene el usuario seleccionado.
            if (usuarioSeleccionado != null) {  // Si hay un usuario seleccionado, actualiza el formulario.
                nombreField.setValue(usuarioSeleccionado.getNombre());  // Establece el nombre del usuario en el campo de texto.
                tipoField.setValue(usuarioSeleccionado.getTipo());  // Establece el tipo del usuario en el ComboBox.
            } else {  // Si no hay usuario seleccionado, limpia los campos.
                clearFields();
            }
        });
    }

    private void configureForm() {  // Método para configurar el formulario.
        tipoField.setItems("cliente", "vendedor");  // Define las opciones del ComboBox para el tipo de usuario.
        tipoField.setRequired(true);  // Establece que el tipo de usuario es un campo obligatorio.
        nombreField.setRequired(true);  // Establece que el nombre del usuario es un campo obligatorio.
    }

    private HorizontalLayout createHeader() {  // Método para crear el encabezado con botones de navegación.
        Button productosButton = new Button("Ir a Productos", e -> {  // Crea un botón para navegar a la vista de productos.
            getUI().ifPresent(ui -> ui.navigate("productos"));  // Navega a la vista "productos" cuando se hace clic en el botón.
        });
        Button ventasButton = new Button("Ir a Ventas", e -> {  // Crea un botón para navegar a la vista de ventas.
            getUI().ifPresent(ui -> ui.navigate("ventas"));  // Navega a la vista "ventas" cuando se hace clic en el botón.
        });
        return new HorizontalLayout(productosButton, ventasButton);  // Devuelve un layout horizontal con los botones de productos y ventas.
    }

    private HorizontalLayout createFormLayout() {  // Método para crear el formulario con los campos de entrada y botones de acción.
        Button addButton = new Button("Agregar Usuario", e -> {  // Crea un botón para agregar un nuevo usuario.
            try {
                // Verifica que el nombre no esté vacío.
                if (nombreField.getValue() == null || nombreField.getValue().trim().isEmpty()) {
                    Notification.show("El nombre no puede estar vacío", 3000, Notification.Position.MIDDLE);  // Muestra una notificación si el nombre está vacío.
                    return;
                }
                // Verifica que el tipo de usuario esté seleccionado.
                if (tipoField.getValue() == null) {
                    Notification.show("Debe seleccionar un tipo de usuario", 3000, Notification.Position.MIDDLE);  // Muestra una notificación si el tipo no está seleccionado.
                    return;
                }

                // Llama al servicio para registrar el usuario.
                usuarioService.registrarUsuario(
                        nombreField.getValue().trim(),  // Nombre del usuario.
                        tipoField.getValue());  // Tipo de usuario.
                refreshGrid();  // Refresca el Grid con los nuevos datos.
                clearFields();  // Limpia los campos del formulario.
                Notification.show("Usuario agregado exitosamente", 3000, Notification.Position.MIDDLE);  // Muestra una notificación de éxito.
            } catch (IllegalArgumentException ex) {  // Captura cualquier excepción relacionada con los datos.
                Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);  // Muestra el mensaje de error.
            }
        });

        Button updateButton = new Button("Modificar Usuario", e -> {  // Crea un botón para modificar un usuario seleccionado.
            if (usuarioSeleccionado != null) {  // Verifica si un usuario está seleccionado.
                try {
                    // Verifica que el nombre no esté vacío.
                    if (nombreField.getValue() == null || nombreField.getValue().trim().isEmpty()) {
                        Notification.show("El nombre no puede estar vacío", 3000, Notification.Position.MIDDLE);  // Muestra una notificación si el nombre está vacío.
                        return;
                    }
                    // Verifica que el tipo de usuario esté seleccionado.
                    if (tipoField.getValue() == null) {
                        Notification.show("Debe seleccionar un tipo de usuario", 3000, Notification.Position.MIDDLE);  // Muestra una notificación si el tipo no está seleccionado.
                        return;
                    }

                    // Actualiza los datos del usuario seleccionado.
                    usuarioSeleccionado.setNombre(nombreField.getValue().trim());  // Actualiza el nombre.
                    usuarioSeleccionado.setTipo(tipoField.getValue());  // Actualiza el tipo.
                    usuarioService.actualizarUsuario(usuarioSeleccionado);  // Llama al servicio para actualizar el usuario.
                    refreshGrid();  // Refresca el Grid con los nuevos datos.
                    clearFields();  // Limpia los campos del formulario.
                    Notification.show("Usuario modificado exitosamente", 3000, Notification.Position.MIDDLE);  // Muestra una notificación de éxito.
                } catch (IllegalArgumentException ex) {  // Captura cualquier excepción relacionada con los datos.
                    Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);  // Muestra el mensaje de error.
                }
            } else {
                Notification.show("Por favor seleccione un usuario para modificar", 3000, Notification.Position.MIDDLE);  // Muestra una notificación si no se selecciona un usuario.
            }
        });

        Button deleteButton = new Button("Eliminar Usuario", e -> {  // Crea un botón para eliminar un usuario seleccionado.
            if (usuarioSeleccionado != null) {  // Verifica si un usuario está seleccionado.
                try {
                    usuarioService.eliminarUsuario(usuarioSeleccionado.getId());  // Llama al servicio para eliminar el usuario.
                    refreshGrid();  // Refresca el Grid con los nuevos datos.
                    clearFields();  // Limpia los campos del formulario.
                    Notification.show("Usuario eliminado exitosamente", 3000, Notification.Position.MIDDLE);  // Muestra una notificación de éxito.
                } catch (IllegalArgumentException ex) {  // Captura cualquier excepción relacionada con los datos.
                    Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);  // Muestra el mensaje de error.
                }
            } else {
                Notification.show("Por favor seleccione un usuario para eliminar", 3000, Notification.Position.MIDDLE);  // Muestra una notificación si no se selecciona un usuario.
            }
        });

        return new HorizontalLayout(nombreField, tipoField, addButton, updateButton, deleteButton);  // Devuelve un layout horizontal con los campos y botones del formulario.
    }

    private void refreshGrid() {  // Método para refrescar el Grid con los datos actuales de los usuarios.
        grid.setItems(usuarioService.listarUsuarios());  // Obtiene la lista de usuarios y la muestra en el Grid.
    }

    private void clearFields() {  // Método para limpiar los campos del formulario.
        nombreField.clear();  // Limpia el campo de nombre.
        tipoField.clear();  // Limpia el campo de tipo.
        usuarioSeleccionado = null;  // Establece que no hay usuario seleccionado.
    }
}


`view/VentaView.java`

In [ ]:
// Importación de clases necesarias
package com.proyecto.view;

import com.proyecto.models.Venta;
import com.proyecto.models.Producto;
import com.proyecto.models.Usuario;
import com.proyecto.service.VentaService;
import com.proyecto.service.ProductoService;
import com.proyecto.service.UsuarioService;
import com.vaadin.flow.component.button.Button;
import com.vaadin.flow.component.combobox.ComboBox;
import com.vaadin.flow.component.grid.Grid;
import com.vaadin.flow.component.notification.Notification;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.component.textfield.IntegerField;
import com.vaadin.flow.router.Route;
import com.vaadin.flow.router.BeforeEnterEvent;
import com.vaadin.flow.router.BeforeEnterObserver;
import org.springframework.stereotype.Component;

// Componente de vista para ventas con la ruta "/ventas"
@Component
@Route(value = "ventas", layout = MainLayout.class)
public class VentaView extends VerticalLayout implements BeforeEnterObserver {

    // Declaración de servicios y componentes de UI
    private final VentaService ventaService;
    private final ProductoService productoService;
    private final UsuarioService usuarioService;
    private ComboBox<Producto> productoField;
    private ComboBox<Usuario> clienteField;
    private ComboBox<Usuario> vendedorField;
    private Grid<Venta> grid;

    // Constructor de la clase, inicializa los servicios y configura la vista
    public VentaView(VentaService ventaService, ProductoService productoService, UsuarioService usuarioService) {
        this.ventaService = ventaService;
        this.productoService = productoService;
        this.usuarioService = usuarioService;

        setSizeFull();  // Establece que el layout ocupe todo el tamaño disponible
        setupView();    // Llama al método para configurar los componentes de la vista
    }

    // Método que se ejecuta antes de ingresar a la vista para actualizar los datos
    @Override
    public void beforeEnter(BeforeEnterEvent event) {
        // Actualiza la lista de productos
        if (productoField != null) {
            productoField.setItems(productoService.listarProductos());
        }
        // Actualiza la lista de clientes filtrando solo aquellos de tipo "cliente"
        if (clienteField != null) {
            clienteField.setItems(usuarioService.listarUsuarios().stream()
                    .filter(u -> u.getTipo().equals("cliente"))
                    .toList());
        }
        // Actualiza la lista de vendedores filtrando solo aquellos de tipo "vendedor"
        if (vendedorField != null) {
            vendedorField.setItems(usuarioService.listarUsuarios().stream()
                    .filter(u -> u.getTipo().equals("vendedor"))
                    .toList());
        }
        // Actualiza la lista de ventas en el grid
        if (grid != null) {
            grid.setItems(ventaService.listarVentas());
        }
    }

    // Método que configura todos los componentes de la vista
    private void setupView() {
        // Inicialización del grid y los campos de formulario
        grid = new Grid<>(Venta.class);
        productoField = new ComboBox<>("Producto");
        clienteField = new ComboBox<>("Cliente");
        vendedorField = new ComboBox<>("Vendedor");
        IntegerField cantidadField = new IntegerField("Cantidad");

        // Configuración del grid: columnas de la venta
        grid.setColumns("id", "producto", "cliente", "vendedor", "cantidad", "fecha");
        grid.getColumns().forEach(col -> col.setAutoWidth(true));  // Hace que las columnas tengan ancho automático

        // Configuración de los campos de formulario
        productoField.setItems(productoService.listarProductos());
        productoField.setItemLabelGenerator(Producto::getNombre);
        productoField.setRequired(true);  // El campo producto es obligatorio

        clienteField.setItems(usuarioService.listarUsuarios().stream()
                .filter(u -> u.getTipo().equals("cliente"))
                .toList());
        clienteField.setItemLabelGenerator(Usuario::getNombre);
        clienteField.setRequired(true);  // El campo cliente es obligatorio

        vendedorField.setItems(usuarioService.listarUsuarios().stream()
                .filter(u -> u.getTipo().equals("vendedor"))
                .toList());
        vendedorField.setItemLabelGenerator(Usuario::getNombre);
        vendedorField.setRequired(true);  // El campo vendedor es obligatorio

        cantidadField.setMin(1);  // La cantidad mínima es 1
        cantidadField.setRequired(true);  // El campo cantidad es obligatorio

        // Creación de botones para las acciones de la vista
        Button productosButton = new Button("Ir a Productos", e -> {
            getUI().ifPresent(ui -> ui.navigate("productos"));  // Redirige a la vista de productos
        });

        Button usuariosButton = new Button("Ir a Usuarios", e -> {
            getUI().ifPresent(ui -> ui.navigate("usuarios"));  // Redirige a la vista de usuarios
        });

        // Botón para registrar una nueva venta
        Button saveButton = new Button("Registrar Venta", e -> {
            try {
                // Validación de los campos del formulario
                if (productoField.getValue() == null || clienteField.getValue() == null ||
                        vendedorField.getValue() == null || cantidadField.getValue() == null) {
                    Notification.show("Por favor complete todos los campos", 3000, Notification.Position.MIDDLE);
                    return;
                }

                // Llamada al servicio para registrar la venta
                ventaService.registrarVenta(
                        productoField.getValue(),
                        clienteField.getValue(),
                        vendedorField.getValue(),
                        cantidadField.getValue());
                // Actualiza la lista de ventas en el grid
                grid.setItems(ventaService.listarVentas());
                clearFields(productoField, clienteField, vendedorField, cantidadField);  // Limpia los campos
                Notification.show("Venta registrada exitosamente", 3000, Notification.Position.MIDDLE);
            } catch (IllegalArgumentException ex) {
                Notification.show(ex.getMessage(), 3000, Notification.Position.MIDDLE);  // Muestra mensaje de error si ocurre
            }
        });

        // Botón para eliminar una venta seleccionada
        Button deleteButton = new Button("Eliminar Venta", e -> {
            Venta selectedVenta = grid.getSelectedItems().stream().findFirst().orElse(null);  // Obtiene la venta seleccionada
            if (selectedVenta != null) {
                ventaService.eliminarVenta(selectedVenta.getId());  // Llama al servicio para eliminar la venta
                grid.setItems(ventaService.listarVentas());  // Actualiza la lista de ventas en el grid
                Notification.show("Venta eliminada exitosamente", 3000, Notification.Position.MIDDLE);
            } else {
                Notification.show("Por favor seleccione una venta para eliminar", 3000, Notification.Position.MIDDLE);  // Si no hay venta seleccionada
            }
        });

        // Botón para actualizar la lista de productos, clientes, vendedores y ventas
        Button refreshButton = new Button("Actualizar Lista", e -> {
            productoField.setItems(productoService.listarProductos());
            clienteField.setItems(usuarioService.listarUsuarios().stream()
                    .filter(u -> u.getTipo().equals("cliente"))
                    .toList());
            vendedorField.setItems(usuarioService.listarUsuarios().stream()
                    .filter(u -> u.getTipo().equals("vendedor"))
                    .toList());
            grid.setItems(ventaService.listarVentas());  // Actualiza la lista de ventas
            Notification.show("Lista actualizada", 3000, Notification.Position.MIDDLE);
        });

        // Crear el layout con los botones
        HorizontalLayout header = new HorizontalLayout(productosButton, usuariosButton);
        HorizontalLayout formLayout = new HorizontalLayout(
                productoField, clienteField, vendedorField, cantidadField,
                saveButton, deleteButton, refreshButton);

        // Agregar los componentes a la vista
        add(header, grid, formLayout);
        grid.setItems(ventaService.listarVentas());  // Inicializa el grid con las ventas
    }

    // Método para limpiar los campos del formulario
    private void clearFields(ComboBox<Producto> productoField, ComboBox<Usuario> clienteField,
            ComboBox<Usuario> vendedorField, IntegerField cantidadField) {
        productoField.clear();
        clienteField.clear();
        vendedorField.clear();
        cantidadField.clear();
    }
}


## Creacion de la Aplicación principal 

`/ProyectoApplication.java`

In [ ]:
// Paquete principal del proyecto
package com.proyecto;

// Importaciones necesarias para la configuración de Spring Boot y Vaadin
import org.springframework.boot.SpringApplication;
import org.springframework.boot.autoconfigure.SpringBootApplication;
import org.springframework.boot.web.servlet.support.SpringBootServletInitializer;
import com.vaadin.flow.spring.annotation.EnableVaadin;

// Anotación que indica que esta clase es la aplicación principal de Spring Boot
@SpringBootApplication
// Anotación que habilita la integración de Vaadin con Spring Boot
@EnableVaadin
public class ProyectoApplication extends SpringBootServletInitializer {

    // Método principal para ejecutar la aplicación Spring Boot
    public static void main(String[] args) {
        // Ejecuta la aplicación Spring Boot
        SpringApplication.run(ProyectoApplication.class, args);
    }
}


## Actualizar AppTest.java 

In [ ]:
package com.proyecto;

import org.junit.jupiter.api.Test;
import static org.junit.jupiter.api.Assertions.*;

/**
 * Unit test for simple App.
 */
public class AppTest {
    /**
     * Rigorous Test :-)
     */
    @Test
    public void shouldAnswerWithTrue() {
        assertTrue(true);
    }
}
